In [1]:
# DL framework
import tensorflow as tf

from datetime import datetime

# common packages
import numpy as np
import os
import sys
import math

# timing epochs
import time

# read image
from scipy.misc import imread

# for ordered dict when building layer components
import collections

# plotting pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot

# for data augmentation
from scipy import ndimage
from scipy import misc

# for shuffling data batches
from sklearn.utils import shuffle

# const
SEED = 42

# Helper to make the output consistent
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

    
# set log level to supress messages, unless an error
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Version information
print("Python: {}".format(sys.version_info[:]))
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU found')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
reset_graph()

Python: (3, 5, 4, 'final', 0)
TensorFlow: 1.4.0
Default GPU Device: /device:GPU:0


In [2]:
# for saving the best params
# this shouldn't be necessary but I'm having trouble using the saver to save
# with the best params
import pickle
def save_obj(obj, name):
    with open('best_params/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('best_params/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
SQUARE_DIM = 224
if SQUARE_DIM:
    IMG_WIDTH = SQUARE_DIM
    IMG_HEIGHT = SQUARE_DIM
    
CHANNELS = 3

# utility plotting function
def plot_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.title("{}x{}".format(image.shape[1], image.shape[0]))
    plt.axis("off")
    
# load dataset into memory
X_tr = np.load('../numpy/sigmoid/' + str(SQUARE_DIM) + '/X_train.npy')
y_tr = np.load('../numpy/sigmoid/' + str(SQUARE_DIM) + '/y_train.npy')

X_val = np.load('../numpy/sigmoid/' + str(SQUARE_DIM) + '/X_val.npy')
y_val = np.load('../numpy/sigmoid/' + str(SQUARE_DIM) + '/y_val.npy')

X_test = np.load('../numpy/sigmoid/' + str(SQUARE_DIM) + '/X_test.npy')
y_test = np.load('../numpy/sigmoid/' + str(SQUARE_DIM) + '/y_test.npy')

In [4]:
# # validation
X_val /= 255
X_val -= 0.5
X_val *= 2


# # test
X_test /= 255
X_test -= 0.5
X_test *= 2

In [5]:
# this function is adapted from https://github.com/ageron/handson-ml/blob/master/13_convolutional_neural_networks.ipynb
def val_augment_image(image, lrf_p=0.0, upf_p=0.0, roll_num=0):
        
    n_image = image

    # Flip the image horizontally with upf_p% probability:
    if np.random.rand() < lrf_p:
        n_image = np.fliplr(n_image)

    # Flip the image vertically with lrf_p% probability:
    if np.random.rand() < upf_p:
        n_image = np.flipud(n_image)
        
    # 50% chance roll image horiz
    if np.random.rand() < 0.5:
        val = int(np.random.rand() * roll_num)
        # 50% left or right
        if np.random.rand() < 0.5:
            n_image = np.roll(n_image, val, axis=1)
        else:
            n_image = np.roll(n_image, -val, axis=1)
        
    # 50% chance roll image vert
    if np.random.rand() < 0.5:
        val = int(np.random.rand() * roll_num)
        # 50% left or right
        if np.random.rand() < 0.5:
            n_image = np.roll(n_image, val, axis=0)
        else:
            n_image = np.roll(n_image, -val, axis=0)

    # ensure float32
    n_image = n_image.astype(np.float32)

    return n_image

In [6]:
# this function is adapted from https://github.com/ageron/handson-ml/blob/master/13_convolutional_neural_networks.ipynb
def augment_image(image, lrf_p=0.0, upf_p=0.0, r_degree=0.0, max_zoom=0.0, roll_num=0):
        
    n_image = image
    
    
    # Flip the image horizontally with upf_p% probability:
    if np.random.rand() < lrf_p:
        n_image = np.fliplr(n_image)

    # Flip the image vertically with lrf_p% probability:
    if np.random.rand() < upf_p:
        n_image = np.flipud(n_image)

    # 50% chance roll image horiz
    if np.random.rand() < 0.5:
        val = int(np.random.rand() * roll_num)
        # 50% left or right
        if np.random.rand() < 0.5:
            n_image = np.roll(n_image, val, axis=1)
        else:
            n_image = np.roll(n_image, -val, axis=1)
        
    # 50% chance roll image vert
    if np.random.rand() < 0.5:
        val = int(np.random.rand() * roll_num)
        # 50% left or right
        if np.random.rand() < 0.5:
            n_image = np.roll(n_image, val, axis=0)
        else:
            n_image = np.roll(n_image, -val, axis=0)

    # normalize on the fly
    # this is inefficient, but we need to use this here since the resize converts
    # back from uint8
    n_image = n_image.astype(np.float32)

    #print("pre", n_image[0][0])
    n_image /= 255
    n_image -= 0.5
    n_image *= 2
    
    if r_degree > 0.0:
        r_degree *= np.random.rand()
        # randomly choose rotation direction
        r_dir = (1 if np.random.rand() > 0.5 else -1)
        n_image = ndimage.interpolation.rotate(n_image, r_dir*r_degree, reshape=False)
    

    return n_image

In [7]:
# this is not my code -- this is adapted from hands on ML
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params, g, sess):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: g.get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    sess.run(assign_ops, feed_dict=feed_dict)

In [8]:
batch_size = 16
n_outputs = 1

# l2_regularizer = tf.contrib.layers.l2_regularizer(0.01)
he_init = tf.contrib.layers.variance_scaling_initializer()

In [9]:
def build_vgg(ARCH_ID, NAME_STR):

    reset_graph()
    g = tf.Graph()
    with g.as_default():

        with tf.name_scope("hyper_params"):
            learning_rate = tf.placeholder(tf.float32, name='learning_rate')

        with tf.name_scope("inputs"):
            # data
            X = tf.placeholder(tf.float32, shape=[None, IMG_HEIGHT, IMG_WIDTH, CHANNELS], name="X")

            # labels
            y_raw = tf.placeholder(tf.int64, shape=[None, n_outputs], name="y_input")
            y_ = tf.cast(y_raw, tf.float32)

            # for training/evaluating
            training = tf.placeholder_with_default(False, shape=[], name="training")

        
        with tf.name_scope("gan"):
            # 224x224
            conv1_1 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=3,
                                   strides=2, padding="SAME",
                                   activation=tf.nn.selu, 
                                   kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                   bias_initializer=tf.zeros_initializer(), trainable=True,
                                   name="discriminator/Gconv_1")

            # 112x112
            conv1_2 = tf.layers.conv2d(conv1_1, 128, kernel_size=3, strides=2, padding='same', 
                                   activation=tf.nn.selu,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                   bias_initializer=tf.zeros_initializer(), trainable=True,
                                   name="discriminator/Gconv_2")

            # 56x56
            conv2_1 = tf.layers.conv2d(conv1_2, 256, kernel_size=3, strides=2, padding='same', 
                                               activation=tf.nn.selu,
                                               kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                               bias_initializer=tf.zeros_initializer(), trainable=True,
                                               name="discriminator/Gconv_3")
            
#             

            # 28x28x256
            conv2_2 = tf.layers.conv2d(conv2_1, 512, kernel_size=3, strides=2, padding='same', 
                                   activation=tf.nn.selu,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                   bias_initializer=tf.zeros_initializer(), trainable=True,
                                   name="discriminator/Gconv_4")
#             
            # 14x14x512
            conv2_3 = tf.layers.conv2d(conv2_2, 768, kernel_size=3, strides=1, padding='same', 
                                   activation=tf.nn.selu,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                   bias_initializer=tf.zeros_initializer(), trainable=True,
                                   name="discriminator/my_conv")
            conv2_2 = tf.layers.max_pooling2d(inputs=conv2_3, pool_size=[2,2], strides=2, name="max_pool_myc")
            #7x7x768
        
            ###### 0
            if NAME_STR == "discrim_class_0c_0f" or NAME_STR == "discrim_class_0c_4f":
                last_shape = int(np.prod(conv2_2.get_shape()[1:]))
                pool_flat = tf.reshape(conv2_2, shape=[-1, last_shape])
        
        
        
            #-------------------------------------------------------------------------------------------------------------
            ##### A

            elif NAME_STR == "discrim_class_1c_0f" or NAME_STR == "discrim_class_1c_4f":
                # 14x14
                conv_new_01 = tf.layers.conv2d(conv2_2, 1024, kernel_size=7, strides=1, padding='same', 
                                           activation=tf.nn.selu,
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                           bias_initializer=tf.zeros_initializer(), trainable=True,
                                           name="new_01")
                last_shape = int(np.prod(conv_new_01.get_shape()[1:]))
                pool_flat = tf.reshape(conv_new_01, shape=[-1, last_shape])
            
            #-------------------------------------------------------------------------------------------------------------
            ##### B
            elif NAME_STR == "discrim_class_2c_0f" or NAME_STR == "discrim_class_2c_4f":
                # 14x14
                conv_new_01 = tf.layers.conv2d(conv2_2, 1024, kernel_size=3, strides=1, padding='same', 
                                           activation=tf.nn.selu,
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                           bias_initializer=tf.zeros_initializer(), trainable=True,
                                           name="new_01")

                conv2_2_pool = tf.layers.max_pooling2d(inputs=conv_new_01, padding='same', pool_size=[2,2], strides=2, name="max_pool_02")

                # 7x7
                conv_new_02 = tf.layers.conv2d(conv2_2_pool, 2048, kernel_size=3, strides=1, 
                                           activation=tf.nn.selu,
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                           bias_initializer=tf.zeros_initializer(), trainable=True,
                                           name="new_02")
                last_shape = int(np.prod(conv_new_02.get_shape()[1:]))
                pool_flat = tf.reshape(conv_new_02, shape=[-1, last_shape])
            
            # ------------------------------------------------------------------------------------------ [END]
            else:
                print("shoot - somethings wrong")

            # reshape
            print("last_shape: ", last_shape)
            pool_flat_drop = tf.layers.dropout(pool_flat, 0.33, training=training, name="entry_drop")
        

#         with tf.name_scope("fc1"):
#             fc1 = tf.layers.dense(pool_flat_drop, 512, kernel_initializer=he_init, activation=tf.nn.selu, name="fc1")
#             fc1_drop = tf.layers.dropout(fc1, 0.33, training=training, name="fc1_drop")

#         with tf.name_scope("fc2"):
#             fc2 = tf.layers.dense(fc1_drop, 256, kernel_initializer=he_init, activation=tf.nn.selu, name="fc2")
#             fc2 = tf.layers.dropout(fc2, 0.5, training=training, name="fc1_drop")

#         with tf.name_scope("fc2"):
            fc1 = tf.layers.dense(pool_flat_drop, 64, kernel_initializer=he_init, activation=tf.nn.selu, name="fc1")

            
        with tf.name_scope("output_layer"):
            logits_ = tf.layers.dense(fc1, n_outputs, name="sigmoid_output")
            preds = tf.sigmoid(logits_, name="preds")

        # ================================================ cost
        with tf.name_scope("cross_entropy"):
            xentropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_, labels=y_)
            batch_loss = tf.reduce_mean(xentropy)

        with tf.name_scope("train"):
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                               beta1=0.9,
                                               beta2=0.999,
                                               epsilon=1e-08,
                                               use_locking=False,
                                               name='Adam')
            
            all_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            if NAME_STR == "discrim_class_0c_0f" or NAME_STR == "discrim_class_1c_0f" or NAME_STR == "discrim_class_2c_0f":
                train_vars = all_vars
                print("training all the vars")
            else:
                freeze_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="discriminator/Gconv_[1234]")
                train_vars = [var for var in all_vars if var not in freeze_vars]
                print("num vars:", len(all_vars), "= frozen(", len(freeze_vars), ")", "+ train(", len(train_vars), ")")

            training_op = optimizer.minimize(batch_loss, var_list=train_vars, name="training_op")


        # =============================================== metrics
        with tf.name_scope("train_metrics") as scope:    
            # train_y_true_cls = tf.cast(y_, tf.bool)
            train_y_true_cls = tf.greater_equal(y_, 0.5)
            train_y_pred_cls = tf.greater_equal(preds, 0.5)

            correct_prediction = tf.equal(train_y_pred_cls, train_y_true_cls)
            train_batch_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
            train_auc, train_auc_update = tf.metrics.auc(labels=y_, predictions=preds)

            train_tf_acc, train_tf_acc_op = tf.metrics.accuracy(labels=train_y_true_cls, predictions=train_y_pred_cls)
            train_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
            train_acc_reset_op = tf.variables_initializer(train_acc_vars, name="train_acc_reset_op")
            
        for node in (y_, preds, train_y_true_cls, train_y_pred_cls, correct_prediction, train_batch_acc):
            g.add_to_collection("label_nodes", node)

        with tf.name_scope("validation_metrics") as scope:    
            val_y_true_cls = tf.greater_equal(y_, 0.5)
            val_y_pred_cls = tf.greater_equal(preds, 0.5)

            val_correct_prediction = tf.equal(val_y_pred_cls, val_y_true_cls)
            val_batch_acc = tf.reduce_mean(tf.cast(val_correct_prediction, tf.float32))
            
            val_auc, val_auc_update = tf.metrics.auc(labels=y_, predictions=preds)

            val_tf_acc, val_tf_acc_op = tf.metrics.accuracy(labels=val_y_true_cls, predictions=val_y_pred_cls)
            val_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
            val_acc_reset_op = tf.variables_initializer(val_acc_vars, name="val_acc_reset_op")

        with tf.name_scope("test_metrics") as scope:    
            test_y_true_cls = tf.greater_equal(y_, 0.5)
            test_y_pred_cls = tf.greater_equal(preds, 0.5)
            
            test_auc, test_auc_update = tf.metrics.auc(labels=y_, predictions=preds)
            
            test_tf_acc, test_tf_acc_op = tf.metrics.accuracy(labels=test_y_true_cls, predictions=test_y_pred_cls)
            test_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
            test_acc_reset_op = tf.variables_initializer(test_acc_vars, name="test_acc_reset_op")

        # =============================================== loss 
        with tf.name_scope("train_loss_eval") as scope:
            train_mean_loss, train_mean_loss_update = tf.metrics.mean(batch_loss)
            train_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
            train_loss_reset_op = tf.variables_initializer(train_loss_vars, name="train_loss_reset_op")

        with tf.name_scope("val_loss_eval") as scope:
            val_mean_loss, val_mean_loss_update = tf.metrics.mean(batch_loss)
            val_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
            val_loss_reset_op = tf.variables_initializer(val_loss_vars, name="val_loss_reset_op")

        with tf.name_scope("test_loss_eval") as scope:
            test_mean_loss, test_mean_loss_update = tf.metrics.mean(batch_loss)
            test_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
            test_loss_reset_op = tf.variables_initializer(test_loss_vars, name="test_loss_rest_op")

        # ================================================ initialize and save
        with tf.name_scope("save_session"):
            init_global = tf.global_variables_initializer()
            init_local = tf.local_variables_initializer()
            saver = tf.train.Saver()


        # ==================================== combine operations
        # ===== epoch, train
        epoch_train_loss_scalar = tf.summary.scalar('train_epoch_loss', train_mean_loss)
#         epoch_train_acc_scalar = tf.summary.scalar('train_epoch_acc', train_tf_acc)
        epoch_train_auc_scalar = tf.summary.scalar('train_epoch_auc', train_auc)
        epoch_train_write_op = tf.summary.merge([epoch_train_loss_scalar, epoch_train_auc_scalar], name="epoch_train_write_op")

        # ===== epoch, validation
        epoch_validation_loss_scalar = tf.summary.scalar('validation_epoch_loss', val_mean_loss)
#         epoch_validation_acc_scalar = tf.summary.scalar('validation_epoch_acc', val_tf_acc)
        epoch_validation_auc_scalar = tf.summary.scalar('validation_epoch_auc', val_auc)
        epoch_validation_write_op = tf.summary.merge([epoch_validation_loss_scalar, epoch_validation_auc_scalar], name="epoch_validation_write_op")

        # ====== batch, train
        train_batch_loss_scalar = tf.summary.scalar('train_batch_loss', batch_loss)
        train_batch_acc_scalar = tf.summary.scalar('train_batch_acc', train_batch_acc)
        train_batch_write_op = tf.summary.merge([train_batch_loss_scalar, train_batch_acc_scalar], name="train_batch_write_op")

        # ====== checkpoint, validation
        checkpoint_validation_loss_scalar = tf.summary.scalar('validation_batch_loss', batch_loss)
        checkpoint_validation_acc_scalar = tf.summary.scalar('validation_batch_acc', val_batch_acc)
        checkpoint_validation_write_op = tf.summary.merge([checkpoint_validation_loss_scalar, checkpoint_validation_acc_scalar], name="checkpoint_valdiation_write_op")
        
        # write operations
        for node in (epoch_train_write_op, epoch_validation_write_op, train_batch_write_op, checkpoint_validation_write_op):
            g.add_to_collection("write_ops", node)
            
        # saver/init
        for node in (init_global, init_local):
            g.add_to_collection("save_init", node)
            
        # acc metrics
        for node in (train_tf_acc, train_tf_acc_op, train_acc_reset_op, 
                     train_auc, train_auc_update,
                     val_tf_acc, val_tf_acc_op, val_acc_reset_op,
                     val_auc, val_auc_update,
                     test_tf_acc, test_tf_acc_op, test_acc_reset_op,
                     test_auc, test_auc_update):
            g.add_to_collection("acc_metrics", node)
        
        # loss metrics
        for node in (train_mean_loss, train_mean_loss_update, train_loss_reset_op, 
                     val_mean_loss, val_mean_loss_update, val_loss_reset_op, 
                     test_mean_loss, test_mean_loss_update, test_loss_reset_op):
            g.add_to_collection("loss_metrics", node)
            
        # test metrics
        for node in (train_y_true_cls, train_y_pred_cls):
            g.add_to_collection("test_metrics", node)
            
        # main operations
        for node in (training_op, X, y_raw, training, learning_rate):
            g.add_to_collection("main_ops", node)

        print("model built.")

    return g, saver

print("ready")

ready


In [10]:
from sklearn.utils import shuffle
def run_that_model(NAME_STR):
    global X_tr
    global y_tr
    global X_val
    global y_val
    global X_test
    global y_test
    global batch_size
    print("batch_size: ", batch_size)
    NUM_EPOCHS = 50
    
    # lr_choice = [1e-5, 1e-4, 1e-3]
    # LEARNING_RATE = np.random.choice(lr_choice).tolist()
#     LEARNING_RATE = 0.000001
    LEARNING_RATE = 1e-6
    # -----------
    #####################################
    # ----------- augmentation
    ##### training
    AUG_BOOL = True
    LRF_P = 0.5
    UPF_p = 0.5
    MAX_ZOOM = 0.0
    ROLL_NUM = 25
    ROT_DEG = 0.0
    
    ##### validation
    VAL_AUG_BOOL = True
    VAL_LRF_P = 0.3
    VAL_UPF_P = 0.3
    VAL_ROLL_NUM = 20
    ######################################

    # --- save best params
    best_model_params = None
    best_loss_val = np.infty

    # ------ for "early-ish" stopping
    # check_interval = 15
    checks_since_last_progress = 0
    # max_checks_without_progress = 20

    # Tensorboard
    now = datetime.now().strftime("%d%b%Y_%H%M%S")
    root_logdir = "tf_logs/final_discrim_class/" + NAME_STR + "_3"
    logdir = "{}/{}/".format(root_logdir, now)

    g, saver = build_vgg(now, NAME_STR)
    
    
    epoch_train_write_op, epoch_validation_write_op, train_batch_write_op, checkpoint_validation_write_op = g.get_collection("write_ops")
    init_global, init_local = g.get_collection("save_init")
    train_tf_acc, train_tf_acc_op, train_acc_reset_op, train_auc, train_auc_update, val_tf_acc, val_tf_acc_op, val_acc_reset_op, val_auc, val_auc_update, test_tf_acc, test_tf_acc_op, test_acc_reset_op, test_auc, test_auc_update = g.get_collection("acc_metrics")
    train_mean_loss, train_mean_loss_update, train_loss_reset_op, val_mean_loss, val_mean_loss_update, val_loss_reset_op, test_mean_loss, test_mean_loss_update, test_loss_reset_op = g.get_collection("loss_metrics")
    training_op, X, y_raw, training, learning_rate = g.get_collection("main_ops")
    #prepared_image_op, input_image = g.get_collection("preprocess")
    step = 0
    with tf.Session(graph=g) as sess:
        init_global.run()
        init_local.run()
        
        if NAME_STR == "discrim_class_0c_0f" or NAME_STR == "discrim_class_1c_0f" or NAME_STR == "discrim_class_2c_0f":
            print("not loading vars")
        else:
            const_lookup = 0
            max_lookup = 5
            conv1_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope="discriminator/Gconv_1")
            conv1_init = sess.run(conv1_vars)
            print(conv1_init[const_lookup][const_lookup][const_lookup][const_lookup][:max_lookup])
            reuse_vars = g.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                               scope="discriminator/Gconv_[1234]") 
            reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
            restore_saver = tf.train.Saver(reuse_vars_dict)
            restore_saver.restore(sess, "../GAN/saver/GAN_Model_3.ckpt")
            conv1_restore = sess.run(conv1_vars)
            print("loaded {} vars".format(len(reuse_vars_dict)))
            print(conv1_restore[const_lookup][const_lookup][const_lookup][const_lookup][:max_lookup])
        
        
        
        sess.run(val_acc_reset_op)
        sess.run(val_loss_reset_op)
        sess.run(train_acc_reset_op)
        sess.run(train_loss_reset_op)

        # tensorboard writer
        #batch_train_writer = tf.summary.FileWriter(os.path.join(logdir, "cnn/batch/train/"))
        #batch_val_writer = tf.summary.FileWriter(os.path.join(logdir, "cnn/batch/val/"))
        epoch_train_writer = tf.summary.FileWriter(os.path.join(logdir, "cnn/epoch/train/"))
        epoch_val_writer = tf.summary.FileWriter(os.path.join(logdir, "cnn/epoch/val/"))
        epoch_train_writer.add_graph(sess.graph)

        low_i = 0
        high_i = 0
        for epoch in range(NUM_EPOCHS):
            step += 1
            EPOCH_START_TIME = time.time()

            # Shuffle the training set (data and label)
            X_tr, y_tr = shuffle(X_tr, y_tr, random_state=42)

            mini_step = 0
            for iteration in range(len(X_tr) // batch_size):
                mini_step += 1

                low_i = iteration*batch_size
                high_i = (iteration+1)*batch_size
                X_batch_raw = X_tr[low_i:high_i]     
                y_batch = y_tr[low_i:high_i]
#                 print(mini_step, ": ", "[",low_i,",", high_i,"]","of",len(X_batch_raw))

                # augment batch
                # TODO: look into factoring class size
                X_batch_aug = np.zeros((X_batch_raw.shape[0], IMG_WIDTH, IMG_HEIGHT, 3), dtype='float32')
                ind = 0
                for img in X_batch_raw:
                    if AUG_BOOL:
#                         X_batch_aug[ind] = sess.run(prepared_image_op, feed_dict={input_image: img})
                        X_batch_aug[ind] = augment_image(np.copy(img), lrf_p=LRF_P, 
                                                         upf_p=UPF_p, r_degree=ROT_DEG, 
                                                         roll_num=ROLL_NUM)
                    else:
                        X_batch_aug[ind] = np.copy(img)
                    ind += 1

                # shuffle batch after augmentation
                #X_tr, y_tr = shuffle(X_batch_aug, y_batch, random_state=42)


                feed_train = {X: X_batch_aug, 
                              y_raw: y_batch, 
                              training: True,
                              learning_rate: LEARNING_RATE}

                # training operation (back prop+)
                sess.run(training_op, feed_dict=feed_train)
                #step += 1

                # update training metrics, acc and loss, (evaluation on current batch)
                sess.run([train_tf_acc_op, train_auc_update, train_mean_loss_update], feed_dict={X: X_batch_aug, 
                                                                                                 y_raw: y_batch,
                                                                                                 training: False})

                # ================================================================================[validation]
                if mini_step % 8 == 0:
                         # ----- validation
                    # run validation 3x w/ (lesser) augmentation
                    for i in range(4):
#                         X_val, y_val = shuffle(X_val, y_val, random_state=42)
                        lower_i = 0
                        higher_i = 0
                        for vitter in range(len(X_val) // batch_size):
                            lower_i = vitter*batch_size
                            higher_i = (vitter+1)*batch_size
                            X_val_batch = X_val[lower_i:higher_i]
                            y_val_batch = y_val[lower_i:higher_i]

                            # augment
                            X_val_batch_aug = np.zeros((X_val_batch.shape[0], IMG_WIDTH, IMG_HEIGHT, 3), dtype='float32')
                            ind = 0
                            for img in X_val_batch:
                                if VAL_AUG_BOOL:
                                    # no normalization happens here
                                    X_val_batch_aug[ind] = val_augment_image(np.copy(img), 
                                                                             lrf_p=VAL_LRF_P, 
                                                                             upf_p=VAL_UPF_P, 
                                                                             roll_num=VAL_ROLL_NUM)
                                else:
                                    X_val_batch_aug[ind] = np.copy(img)
                                ind += 1

                            sess.run([val_tf_acc_op, val_auc_update, val_mean_loss_update], 
                                     feed_dict={X: X_val_batch_aug, y_raw: y_val_batch, training: False})

                        X_val_batch = X_val[higher_i:]
                        y_val_batch = y_val[higher_i:]
                        # not augmenting final validation batch *(which will be different each round
                        # becuase of the shuffle)
                        if len(X_batch_raw) > 0:
                            sess.run([val_tf_acc_op, val_auc_update, val_mean_loss_update], 
                                     feed_dict={X: X_val_batch, y_raw: y_val_batch, training: False})
            # ===============================================================================[validation end]
            
            # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx Train on remaining batch, if it exists 
            # metrics aren't calculated since the batch size could be really small
            X_batch_raw = X_tr[high_i:]
            y_batch = y_tr[high_i:]

            if len(X_batch_raw) > 0:
#                 print(len(X_batch_raw))
                X_batch_augx = np.zeros((X_batch_raw.shape[0], IMG_WIDTH, IMG_HEIGHT, 3), dtype='float32')
                ind = 0
                for img in X_batch_raw:
                    if AUG_BOOL:
#                         X_batch_aug[ind] = sess.run(prepared_image_op, feed_dict={input_image: img})
                        X_batch_augx[ind] = augment_image(np.copy(img), lrf_p=LRF_P, 
                                                          upf_p=UPF_p, r_degree=ROT_DEG, 
                                                          roll_num=ROLL_NUM)
                    else:
                        X_batch_augx[ind] = np.copy(img)
                    ind += 1

                feed_train = {X: X_batch_augx, 
                              y_raw: y_batch, 
                              training: True,
                              learning_rate: LEARNING_RATE}

                sess.run(training_op, feed_dict=feed_train)
                #step += 1
            # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

            # ========================================================= Epoch metrics (log and print)

            # ----- training
            epoch_acc_print, epoch_auc_print, epoch_loss_print = sess.run([train_tf_acc, train_auc, train_mean_loss])

            epoch_train_summary = sess.run(epoch_train_write_op)
            epoch_train_writer.add_summary(epoch_train_summary, step)
            epoch_train_writer.flush()

            epoch_val_acc_print, epoch_val_auc_print, epoch_val_loss_print = sess.run([val_tf_acc, val_auc, val_mean_loss])

            epoch_val_summary = sess.run(epoch_validation_write_op)
            epoch_val_writer.add_summary(epoch_val_summary, step)
            epoch_val_writer.flush()

            print("E:{}tr: auc {:.2f}%|acc {:.2f}%|{:.4f} || val: auc {:.2f}%|acc {:.2f}%|{:.4f}".format(epoch,
                                                                                             epoch_auc_print*100,              
                                                                                             epoch_acc_print*100, 
                                                                                             epoch_loss_print,
                                                                                             epoch_val_auc_print*100,
                                                                                             epoch_val_acc_print*100, 
                                                                                             epoch_val_loss_print))
            #plot_color_image(X_batch_aug[0])
            #plt.imshow(X_batch_aug[0].astype(np.uint8),interpolation="nearest")
            #plt.title("{}x{}".format(image.shape[1], image.shape[0]))
#             show_output(sess, 9, X_batch_aug, 3, "RGB")
#             plt.axis("off")
            # ==========================================================================================

            # save 'best' model (lowest epoch avg for validation loss)
            if epoch_val_loss_print < best_loss_val:
                best_loss_val = epoch_val_loss_print
                checks_since_last_progress = 0
                best_model_params = get_model_params()
                print("Best params saved| val loss: {:.4f}".format(best_loss_val))
                
            else:
                checks_since_last_progress += 1
            
            if checks_since_last_progress >= 150:
                print("Early Stopping")
                break

            # estimate time remaining
            EPOCH_END_TIME = time.time()
            EPOCH_ELAPSED_TIME = EPOCH_END_TIME - EPOCH_START_TIME
            EST_TIME_REMAINING = (((EPOCH_ELAPSED_TIME) * (NUM_EPOCHS - epoch)) / 60)
            print("E time {:.1f} secs, Est.Time Remain: {:.1f} mins".format(EPOCH_ELAPSED_TIME, EST_TIME_REMAINING))
            print("-----------------------------")

            sess.run(val_acc_reset_op)
            sess.run(val_loss_reset_op)
            sess.run(train_acc_reset_op)
            sess.run(train_loss_reset_op)

        # done with training writers
        #batch_train_writer.close()
        #batch_val_writer.close()
        epoch_train_writer.close()
        epoch_val_writer.close()

        # save session so we can re-enter at a later time
        # restore best params
        restore_model_params(best_model_params, g, sess)
        print("best params restored")
        
        saver_path = "./corrected_saver/" + NAME_STR + "_" + str(SQUARE_DIM) + "_" + "FINAL" + "_448"
        save_path = saver.save(sess, saver_path)
        save_obj(best_model_params, NAME_STR)
        print("best params saved")
        
        return g, best_model_params, saver

print("ready")

ready


In [11]:
g = None
bmp = None
saver = None
sess = None

# run graph
START_TIME = time.time()
# 0
# name_str = "discrim_class_0c_0f"
# name_str = "discrim_class_0c_4f"
# 1
# name_str = "discrim_class_1c_0f"
# name_str = "discrim_class_1c_4f"
# 2
# name_str = "discrim_class_2c_0f"
name_str = "discrim_class_2c_4f"

g, bmp, saver = run_that_model(NAME_STR=name_str)
END_TIME = time.time()

# time remaining metrics
print("Time: {:.1f}min".format((END_TIME - START_TIME)/60))

batch_size:  16
last_shape:  8192
num vars: 18 = frozen( 6 ) + train( 12 )
model built.
[ 0.04795146  0.06462063  0.04433362 -0.07806579  0.06958175]
INFO:tensorflow:Restoring parameters from ../GAN/saver/GAN_Model_3.ckpt
loaded 6 vars
[-0.04778407  0.03699316  0.0329456   0.02024674 -0.04461705]
E:0tr: auc 58.56%|acc 56.08%|0.6849 || val: auc 57.54%|acc 56.12%|0.6882
Best params saved| val loss: 0.6882
E time 27.4 secs, Est.Time Remain: 22.9 mins
-----------------------------
E:1tr: auc 60.54%|acc 58.08%|0.6771 || val: auc 60.77%|acc 59.21%|0.6794
Best params saved| val loss: 0.6794
E time 26.5 secs, Est.Time Remain: 21.7 mins
-----------------------------
E:2tr: auc 60.21%|acc 56.50%|0.6764 || val: auc 61.96%|acc 60.85%|0.6750
Best params saved| val loss: 0.6750
E time 26.6 secs, Est.Time Remain: 21.2 mins
-----------------------------
E:3tr: auc 62.74%|acc 59.42%|0.6687 || val: auc 62.99%|acc 61.60%|0.6708
Best params saved| val loss: 0.6708
E time 26.3 secs, Est.Time Remain: 20.6 m

KeyboardInterrupt: 

## Eval

In [12]:
reset_graph()
NAME_STR = name_str
g, saver = build_vgg("now", NAME_STR)

saver_path = "./corrected_saver/" + NAME_STR + "_" + str(SQUARE_DIM) + "_" + "FINAL"
sess = tf.Session(graph=g)
saver.restore(sess, saver_path)

training_op, X, y_raw, training, learning_rate = g.get_collection("main_ops")
train_tf_acc, train_tf_acc_op, train_acc_reset_op, train_auc, train_auc_update, val_tf_acc, val_tf_acc_op, val_acc_reset_op, val_auc, val_auc_update, test_tf_acc, test_tf_acc_op, test_acc_reset_op, test_auc, test_auc_update = g.get_collection("acc_metrics")
train_y_true_cls, train_y_pred_cls = g.get_collection("test_metrics")
train_mean_loss, train_mean_loss_update, train_loss_reset_op, val_mean_loss, val_mean_loss_update, val_loss_reset_op, test_mean_loss, test_mean_loss_update, test_loss_reset_op = g.get_collection("loss_metrics")

num_classes = 2

# todo: there is a better way to implement the confusion matrix
# see here; https://stackoverflow.com/questions/41617463/tensorflow-confusion-matrix-in-tensorboard#42857070

with sess:
    confusion_mat = tf.Variable( tf.zeros([num_classes,num_classes], dtype=tf.int32 ), name='confusion')
    sess.run(tf.global_variables_initializer())

    # restore to the best parameters to calculate test metrics
    BMPs = load_obj(NAME_STR)
    if BMPs:
        print("BMP!!!!!")
        restore_model_params(BMPs, g, sess)

    sess.run(test_acc_reset_op)
    sess.run(test_loss_reset_op)
    
    for iteration in range(len(X_test) // batch_size):
        low_i = iteration*batch_size
        high_i = (iteration+1)*batch_size
        X_test_batch = X_test[low_i:high_i]
        y_test_batch = y_test[low_i:high_i]
        sess.run([test_tf_acc_op, test_mean_loss_update, test_auc_update], 
                 feed_dict={X: X_test_batch, y_raw: y_test_batch, training: False})
        jj = train_y_true_cls.eval(feed_dict={y_raw: y_test_batch})
        batch_conf_matrix = tf.confusion_matrix(labels = train_y_true_cls.eval(feed_dict={y_raw: y_test_batch}).reshape(-1),
                                                predictions = train_y_pred_cls.eval(feed_dict={X: X_test_batch}).reshape(-1),
                                                num_classes=num_classes)
        sess.run(confusion_mat.assign(confusion_mat.eval() + batch_conf_matrix.eval()))
    
    X_test_batch = X_test[high_i:]
    y_test_batch = y_test[high_i:]
    if len(X_test_batch) > 0:
        sess.run([test_tf_acc_op, test_mean_loss_update, test_auc_update], 
                 feed_dict={X: X_test_batch, y_raw: y_test_batch, training: False})
        batch_conf_matrix = tf.confusion_matrix(labels = train_y_true_cls.eval(feed_dict={y_raw: y_test_batch}).reshape(-1),
                                                predictions = train_y_pred_cls.eval(feed_dict={X: X_test_batch}).reshape(-1),
                                                num_classes=num_classes)
        sess.run(confusion_mat.assign(confusion_mat.eval() + batch_conf_matrix.eval()))
    final_test_acc, final_test_loss, final_test_auc = sess.run([test_tf_acc, test_mean_loss, test_auc])

    print(">>>>>>>>>> test auc: {:.3f}% acc: {:.2f} loss: {:.5f}".format(final_test_auc*100, 
                                                                         final_test_acc*100,
                                                                         final_test_loss))

    print(confusion_mat.eval())

sess.close()

last_shape:  8192
training all the vars
model built.
INFO:tensorflow:Restoring parameters from ./corrected_saver/discrim_class_2c_0f_224_FINAL
BMP!!!!!
>>>>>>>>>> test auc: 66.831% acc: 59.14 loss: 0.63048
[[64 71]
 [34 88]]


In [1]:
# num = 0
# image = X_tr[num]
# # with tf.Session() as sess:
# #     image = sess.run(prepared_image_op, feed_dict={input_image: image})
# image = augment_image(image=np.copy(image), lrf_p=0.5, max_zoom=0.4, r_degree=0.0, upf_p=0.0)
# print(image[100][100])
# label = y_tr[num]
# print(label)
# plt.imshow(image.astype(np.uint8),interpolation="nearest")
# plt.title("{}x{}".format(image.shape[1], image.shape[0]))
# plt.axis("off")